#### 0. saspy

In [22]:
import saspy
import numpy as np
import pandas as pd
from IPython.display import HTML

#### 1. Config

In [24]:
sascfg="c:/code/pub/ct/sascfg.py"
sas=saspy.SASsession(cfgfile=sascfg)
sas.HTML_Style="PowerPointDark"
_=sas.submit("""
%let ROOT=/home/u63544628;
libname CUBE "/home/u63544628/cubedemo";
""")

Using SAS Config named: oda
SAS Connection established. Subprocess id is 13816



#### 2. Manipulating

In [63]:
resp=sas.submit("""
title font="Consolas" height=20pt bold "BL";

proc import datafile="&ROOT./bl.csv" out=bl dbms=csv replace;
    getnames=yes; guessingrows=300;
    run;

data bl_; set bl;
    rename RESULT=SCORE;
    format TYPE $3. RESULT 1. TYPEX $10.;
    INPUT=trim(OUTPUT);
    OUTPUT=trim(OUTPUT);
    RESULT=put(substr(RESULT,1,1),1.);

    NAN=cmiss(of BATCH--RESULT);

    if BATCH="MAS" then TYPE="MAC";
    else if BATCH="AAS" then TYPE=.;
    else TYPE="#NA";

    TYPEX=put(cats(BATCH,"-",LOCALE,"-",TYPE),$10.);

    SINCE=intck("day",DATE,today());

    if cmiss(of _all_) then delete;

    run;

data bl_; set bl_; keep BATCH INPUT OUTPUT SCORE DATE NAN SINCE; run;

proc sort data=bl_ out=bl_; by INPUT; run;

data bl_; merge bl_(in=x) bl(in=y); by INPUT; if x; run;

data bl_; set bl_; drop BATCH; IDX="bl"||left(put(_N_,10.)); run;

proc print data=bl_(obs=5) noobs; run;
""")
HTML(resp["LST"])

INPUT,OUTPUT,DATE,SCORE,NAN,SINCE,IDX
#1 Companion For Fortnit,#1 Companion For Fortnite,2023-04-15,4,0,141,bl1
#DRIVE,#DRIVE,2023-05-12,2,0,114,bl2
#DRIVE,#DRIVE,2022-07-18,5,0,412,bl3
#VIP Access Weekly,#VIP Access Weekly,2023-05-12,3,0,114,bl4
&ldquo;그거 들었어?,&ldquo;그거 들었어?&rdquo;라고 먼저 말하기,2023-03-29,2,0,158,bl5


In [113]:
resp=sas.submit("""
title font="Consolas" height=20pt bold "CUBEDEMO";

proc format;
	value BOOL .="False" 1="True";
	value YN .="none" 1="Yes" 2="No";
	value SEX .="none" 1="Nam" 2="Yeo";
	run;

data DM; set CUBE.DM; keep SUBJID AGE SEX;
	SUBJID=compress(translate(SUBJID,"","-")); 
    DMIDX="DM"||left(_N_);
	run;

data CM; set CUBE.CM;
    format CMONGO BOOL. CMSTATUS $10.;
    keep CMIDX SUBJID CMTRT CMINDCAE CMONGO CMSTATUS;
    where not missing(CMINDCAE);
    SUBJID=compress(translate(SUBJID,"","-"));
    CMIDX="CM"||left(_N_);

    if CMONGO=. then do; CMSTATUS="none"; end;
    if CMONGO=1 then do; CMSTATUS="having"; end;

    CMINDCAE=substr(CMINDCAE,find(CMINDCAE,"^","i")+1);

    array varchar{*} _character_;
    do q=1 to dim(varchar); varchar{q}=trim(upcase(varchar{q})); end;
    run;

data CM_; retain CMIDX SUBJID SEX AGE CMTRT CMINDCAE; set CM;
    format SEX SEX.;
    merge CM(in=x) DM(in=y); if x;
    run;

proc sort data=CM_ out=CM_; by SUBJID; run;

proc transpose data=CM_ out=CMT; run;
proc print data=CMT; run;

proc transpose data=CM_ out=CM_(drop=_NAME_);
    by SUBJID;
    id SEX;
    var AGE;
    run;

proc print data=CM_(obs=100) noobs; run;
""")
print(resp["LOG"].replace("  ","")[-650:-150])
HTML(resp["LST"])

=1 to dim(varchar); varchar{q}=trim(upcase(varchar{q})); end;
3729 run;
3730 
3731 data CM_; retain CMIDX SUBJID SEX AGE CMTRT CMINDCAE; set CM;
3732 format SEX SEX.;
3733 merge CM(in=x) DM(in=y); if x;
3734 run;
3735 
3736 proc sort data=CM_ out=CM_; by SUBJID; run;
3737 
3738 proc transpose data=CM_ out=CMT; run;
3739 proc print data=CMT; run;
3740 
3741 proc transpose data=CM_ out=CM_(drop=_NAME_);
3742 by SUBJID;
3743 id SEX;
3744 var AGE;
3745 run;
3746 
3747 proc print data=CM_(obs=100) no
